# What's the state of indoor agriculture in the US?

Having worked in food distribution for a decade, I probably spend a lot more time thinking about the logistics of food than most people. When I eat, it's hard for me *not* to imagine where my food came from and how it reached me. Still, it wasn't until the pandemic that I'd ever seriously considered the possibility of losing access to fresh food. The combination of fear and abundant free time is powerful. My long dormant interest in gardening reawakened. I did find some comfort in the ability to grow a few vegetables at home, but it was the knowledge that food was being grown right here in this crowded city that really set my mind at ease. Indoor farms became for me a symbol of hope. Sure the world was a mess, but we might just be able to dig ourselves out of it.

Once I became attuned, it seemed like indoor farms began popping up left and right. Apparently, the industry was quite a bit larger than I'd realized. Looking to understand the landscape better, the question of *where* exactly these farms were located seemed like a good place to start.

## Investigating population density and the emergence of indoor farming
So, what kinds of places become home to indoor farms? There are a lot of ways to characterize a place, so I decided to start by narrowing my scope to population density. 

The [US Decennial Census of Population and Housing](https://www.census.gov/programs-surveys/decennial-census/data/datasets.html) provides a rich source of raw data, but it is challenging to decipher and synthesize into usable features. To begin, I've downloaded raw population demographic data from the [2020 US Census](https://www.census.gov/data/datasets/2020/dec/2020-census-redistricting-summary-file-dataset.html). Deciphering and translating the data required multiple steps. The notebook paragraphs below outline the initial steps of my journey applying data analysis techniques towards my goal of contributing to the understanding of indoor farming and food security overall. 

GIS data for the census tract level, including geographic identifier codes, names, area measurements, and representative latitude and longitude coordinates, is provided in the [National Census Tracts Gazetteer Files](https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.2020.html), listed under the heading "Census Tracts".

### 1. Joining US Census population data with GIS data to display population density

In [1]:
# data files are stored compressed to save time and space
import tarfile
import pandas as pd

# 32mb+ of census data saved in a 4.7mb archive
census_data_archive = "../data_archive/census_data_2022_03_01.tgz"

# 17mb of GIS data saved in a 2.4mb archive
gis_file = "2020_Gaz_tracts_national.gz"

# This is the US Census file with population data we will extract
census_2020_file = "DECENNIALPL2020.P1_data_with_overlays_2021-12-02T121459.csv"

# This extracts a DataFrame from a tgz archived file
def extract_from_tgz(filename):
    with tarfile.open(filename) as tf:
        for file in tf.getmembers():
            if file.name == census_2020_file:
                data = tf.extractfile(file)
                return pd.read_csv(data, low_memory=False)
                        
df_census = extract_from_tgz(census_data_archive)

# change some options that determine how much data is displayed in the notebook
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

Using .shape, we can tell before even setting eyes on it that our DataFrame is going to be enormous - 73 columns and 85,000+ rows!

In [2]:
df_census.shape

(85396, 73)

We can get a pretty good sense of the data just by viewing the first few rows of our DataFrame. So, let's use .head(), to limit the display to the first 5 rows.

In [3]:
df_census.head()

,GEO_ID,NAME,P1_001N,P1_002N,P1_003N,P1_004N,P1_005N,P1_006N,P1_007N,P1_008N,...,P1_062N,P1_063N,P1_064N,P1_065N,P1_066N,P1_067N,P1_068N,P1_069N,P1_070N,P1_071N
0,id,Geographic Area Name,!!Total:,!!Total:!!Population of one race:,!!Total:!!Population of one race:!!White alone,!!Total:!!Population of one race:!!Black or African American alone,!!Total:!!Population of one race:!!American Indian and Alaska Native alone,!!Total:!!Population of one race:!!Asian alone,!!Total:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone,!!Total:!!Population of one race:!!Some Other Race alone,...,!!Total:!!Population of two or more races:!!Population of four races:!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of five races:!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,!!Total:!!Population of two or more races:!!Population of six races:,!!Total:!!Population of two or more races:!!Population of six races:!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775,1653,1389,213,5,8,3,35,...,0,0,0,0,0,0,0,0,0,0
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055,1984,842,1104,2,12,4,20,...,0,0,0,0,0,0,0,0,0,0
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216,3039,2244,714,12,14,6,49,...,0,0,0,0,0,0,0,0,0,0
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4246,3993,3578,327,17,32,1,38,...,0,0,0,0,0,0,0,0,0,0


The population data columns have been named using Census Bureau codes that aren't very meaningful to us. In a moment, we'll rename them using the descriptions provided in the first row (index 0). But first, let's use regex to remove all of those distracting exclamation points! 

In [4]:
df_census.iloc[0].replace('!!', ' ', regex=True, inplace=True)
df_census.iloc[0] = df_census.iloc[0].str.lstrip()

Now we can go ahead and replace the column names with the cleaned up descriptions.

In [5]:
df_census.columns = df_census.iloc[0]
df_census = df_census.drop(0)
df_census

,id,Geographic Area Name,Total:,Total: Population of one race:,Total: Population of one race: White alone,Total: Population of one race: Black or African American alone,Total: Population of one race: American Indian and Alaska Native alone,Total: Population of one race: Asian alone,Total: Population of one race: Native Hawaiian and Other Pacific Islander alone,Total: Population of one race: Some Other Race alone,...,Total: Population of two or more races: Population of four races: American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races:,Total: Population of two or more races: Population of five races: White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander,Total: Population of two or more races: Population of five races: White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race,Total: Population of two or more races: Population of five races: White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races: White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races: White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of five races: Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race,Total: Population of two or more races: Population of six races:,Total: Population of two or more races: Population of six races: White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775,1653,1389,213,5,8,3,35,...,0,0,0,0,0,0,0,0,0,0
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055,1984,842,1104,2,12,4,20,...,0,0,0,0,0,0,0,0,0,0
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216,3039,2244,714,12,14,6,49,...,0,0,0,0,0,0,0,0,0,0
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4246,3993,3578,327,17,32,1,38,...,0,0,0,0,0,0,0,0,0,0
5,1400000US01001020501,"Census Tract 205.01, Autauga County, Alabama",4322,4055,3241,632,29,93,2,58,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85391,1400000US72153750501,"Census Tract 7505.01, Yauco Municipio, Puerto Rico",3968,1967,707,180,35,0,0,1045,...,0,0,0,0,0,0,0,0,0,0
85392,1400000US72153750502,"Census Tract 7505.02, Yauco Municipio, Puerto Rico",1845,915,380,48,10,6,0,471,...,0,0,0,0,0,0,0,0,0,0
85393,1400000US72153750503,"Census Tract 7505.03, Yauco Municipio, Puerto Rico",2155,1152,492,76,7,0,1,576,...,0,0,0,0,0,0,0,0,0,0
85394,1400000US72153750601,"Census Tract 7506.01, Yauco Municipio, Puerto Rico",4368,2221,782,204,23,1,0,1211,...,0,0,0,0,0,0,0,0,0,0


Now that the names for each column describe their contents, we're able to see that we really only need the first column of population data. That means we can "throw away" the other 70 columns! So, let's create a new, much smaller DataFrame with just the columns we need. We'll call it *df_census_pop*.

In [6]:
df_census_pop = pd.DataFrame(df_census[['id', 'Geographic Area Name', 'Total:']])
#df_census = None
df_census_pop

,id,Geographic Area Name,Total:
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4246
5,1400000US01001020501,"Census Tract 205.01, Autauga County, Alabama",4322
...,...,...,...
85391,1400000US72153750501,"Census Tract 7505.01, Yauco Municipio, Puerto Rico",3968
85392,1400000US72153750502,"Census Tract 7505.02, Yauco Municipio, Puerto Rico",1845
85393,1400000US72153750503,"Census Tract 7505.03, Yauco Municipio, Puerto Rico",2155
85394,1400000US72153750601,"Census Tract 7506.01, Yauco Municipio, Puerto Rico",4368


There's still room to improve our column names. Let's make them more descriptive. We'll specify in the first column header that these are "full" GEOIDs, as opposed to the shorter versions we'll be seeing shortly.

In [7]:
df_census_pop.rename(columns={
    'id': 'GEOID Full',
    'Geographic Area Name': 'Census Tract Name',
    'Total:': 'Population'},
    inplace=True)
df_census_pop.head()

,GEOID Full,Census Tract Name,Population
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4246
5,1400000US01001020501,"Census Tract 205.01, Autauga County, Alabama",4322


Let's run .info() on our new DataFrame to see what data oddities might need some attention.

In [8]:
df_census_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85395 entries, 1 to 85395
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GEOID Full         85395 non-null  object
 1   Census Tract Name  85395 non-null  object
 2   Population         85395 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB


According to .info, our columns are free of null values. This sounds a little suspicious... So, let's probe a bit more to see if this is really the case.

We can start by using isna() to identify possible missing values. This will return a boolean indicating where the observation in that column is missing (True) or not (False). 

In [9]:
df_census_pop.isna()

,GEOID Full,Census Tract Name,Population
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
5,False,False,False
...,...,...,...
85391,False,False,False
85392,False,False,False
85393,False,False,False
85394,False,False,False


All of the rows displayed so far are returning False (no missing values). So far, so good. Let's use .sum() to get a count of missing values for our full dataset, to make sure there aren't any we're just not seeing in the current display.

In [10]:
df_census_pop.isna().sum()

0
GEOID Full           0
Census Tract Name    0
Population           0
dtype: int64

Good - still seeing 0 missing values. But, let's check another way to be extra sure. This time, we'll ask pandas to actually show us any rows with missing values. 

In [11]:
df_census_pop[df_census_pop.isna().any(axis=1)]

,GEOID Full,Census Tract Name,Population


Again, nothing! So, it looks like we are in the clear. No missing values to address.

In [12]:
df_census_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85395 entries, 1 to 85395
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   GEOID Full         85395 non-null  object
 1   Census Tract Name  85395 non-null  object
 2   Population         85395 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB


We've learned from .info that all of our columns are of datatype object. Based on visual inspection using. head(), we know that the values in both columns "GEOID Full" and "Census Tract Name" are actually strings, while values in the column "Population" are integers. Let's see if using .astype() to specify types will change what we see when we again run .dtypes.

In [13]:
df_census_pop['GEOID Full'] = df_census_pop['GEOID Full'].astype(str)
df_census_pop['Census Tract Name'] = df_census_pop['Census Tract Name'].astype(str)
df_census_pop['Population'] = df_census_pop['Population'].astype(int)
df_census_pop.dtypes

0
GEOID Full           object
Census Tract Name    object
Population            int32
dtype: object

Good! Population is now showing up correctly as an integer. "GEOID Full" and "Census Tract Name" are still showing up as objects, but this okay. 

We're now ready to load our dataset containing census tract area measurements and lattitude and longitude coordinates. The Census Bureau refers to these as "Gazetteer Files", so we'll call our new DataFrame *df_gaz*. We'll be matching it up shortly with our first DataFrame, *df_census_pop*, so we can calculate population density for each census tract. Since we took a peek at the data before loading it and have also spent HOURS on the Census website, we know that the GEOIDS we are looking at here are for the census tract level. So, we're going to rename our columns to make this very clear.

(Per the Census Bureau, the [GEOID Structure](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html) for a census tract is: STATE+COUNTY+TRACT | Digits: 2+3+6=11)

In [14]:
file_gaz = "../data_archive/2020_Gaz_tracts_national.gz"
df_gaz = pd.read_csv(file_gaz, delimiter = '\t', compression='gzip', dtype={'GEOID' : str})
df_gaz.rename(columns={'GEOID' : 'GEOID Census Tract'}, inplace=True)
df_gaz = df_gaz.round(decimals=2)

Let's check .dtypes on our new DataFrame to make sure the columns we'll be using are of the correct types. We want "GEOID TRACT" to be strings and "ALAND_SQMI" as floating point numbers.

In [15]:
df_gaz.dtypes

USPS                                                                                                                                       object
GEOID Census Tract                                                                                                                         object
ALAND                                                                                                                                       int64
AWATER                                                                                                                                      int64
ALAND_SQMI                                                                                                                                float64
AWATER_SQMI                                                                                                                               float64
INTPTLAT                                                                                                                    

Good. "ALAND_SQMI" is of type float64, which is what we hoped to see. "GEOID TRACT" is of type object, but this *probably* means it is as string, which we want. To make sure, let's see if using .astype to specify that it is a string yields different results when we run .info().

In [16]:
df_gaz['GEOID Census Tract'] = df_gaz['GEOID Census Tract'].astype(str)
df_gaz.rename(columns={'USPS': 'State', 'ALAND_SQMI': 'Land Area Sq Miles', }, inplace=True)
df_gaz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85395 entries, 0 to 85394
Data columns (total 8 columns):
 #   Column                                                                                                                                  Non-Null Count  Dtype  
---  ------                                                                                                                                  --------------  -----  
 0   State                                                                                                                                   85395 non-null  object 
 1   GEOID Census Tract                                                                                                                      85395 non-null  object 
 2   ALAND                                                                                                                                   85395 non-null  int64  
 3   AWATER                                                        

"GEOID Tract" is still showing up as datatype object. Again, since we know it's a string, thi is probably okay. We'll know if anything is amiss if our leading 0s disappear. (But, let's make a note to investigate this question more later...)

We're almost ready for our matchup, we just need to add a new column for "GEOID Tract" to our *df_census_pop* DataFrame, so we can use it to match with our *df_gaz* DataFrame. We know that the last 11 digits of the "GEOID Full" are the GEOIDS for the census tract level. So, we'll create the new "GEOID Tract" column using a lambda to slice just those last 11 digits from "GEOID Full". 

In [17]:
df_census_pop['GEOID Census Tract'] = df_census_pop["GEOID Full"].apply(lambda x: str(x)[-11:])
df_census_pop.head()

,GEOID Full,Census Tract Name,Population,GEOID Census Tract
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775,01001020100
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055,01001020200
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216,01001020300
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4246,01001020400
5,1400000US01001020501,"Census Tract 205.01, Autauga County, Alabama",4322,01001020501


Our new "GEOID Tract" column looks good, so now we can go ahead with the join. We'll join our *df_census_pop* and *df_gaz* DataFrames on the "GEOID Tract" column.

In [18]:
#df_census_pop.join(df_gaz.set_index('GEOID Tract'), on="GEOID Tract").head(10)

In [19]:
df_census_pop = df_census_pop.join(df_gaz.set_index('GEOID Census Tract'), on='GEOID Census Tract')
df_census_pop.head(10)

,GEOID Full,Census Tract Name,Population,GEOID Census Tract,State,ALAND,AWATER,Land Area Sq Miles,AWATER_SQMI,INTPTLAT,INTPTLONG
1,1400000US01001020100,"Census Tract 201, Autauga County, Alabama",1775,01001020100,AL,9825304,28435,3.79,0.01,32.48,-86.49
2,1400000US01001020200,"Census Tract 202, Autauga County, Alabama",2055,01001020200,AL,3320818,5669,1.28,0.00,32.48,-86.47
3,1400000US01001020300,"Census Tract 203, Autauga County, Alabama",3216,01001020300,AL,5349271,9054,2.06,0.00,32.47,-86.46
4,1400000US01001020400,"Census Tract 204, Autauga County, Alabama",4246,01001020400,AL,6384282,8408,2.46,0.00,32.47,-86.44
5,1400000US01001020501,"Census Tract 205.01, Autauga County, Alabama",4322,01001020501,AL,6203654,0,2.40,0.00,32.45,-86.42
6,1400000US01001020502,"Census Tract 205.02, Autauga County, Alabama",3284,01001020502,AL,2097390,379,0.81,0.00,32.47,-86.42
7,1400000US01001020503,"Census Tract 205.03, Autauga County, Alabama",3616,01001020503,AL,3107823,43155,1.20,0.02,32.48,-86.43
8,1400000US01001020600,"Census Tract 206, Autauga County, Alabama",3729,01001020600,AL,8041611,59779,3.10,0.02,32.45,-86.48
9,1400000US01001020700,"Census Tract 207, Autauga County, Alabama",3409,01001020700,AL,22411848,772012,8.65,0.30,32.43,-86.44
10,1400000US01001020801,"Census Tract 208.01, Autauga County, Alabama",3143,01001020801,AL,124272664,8117631,47.98,3.13,32.42,-86.53


Before we go any further, let's create a new DataFrame containing just the columns we want. While we're at it, let's also improve our column names.

In [20]:
df_census_pop = df_census_pop[['GEOID Census Tract', 'State', 'Census Tract Name', 'Population', 'Land Area Sq Miles']].copy()
df_census_pop['Population Density'] = df_census_pop['Population']/df_census_pop['Land Area Sq Miles'].astype(float)
df_census_pop = df_census_pop.round(decimals=2)
#df_census_pop.index = [x for x in range(1, len(df_census_pop.values))]
#df_census_pop[df_census_pop['State'] == 'NY']
df_census_pop[df_census_pop['State'] == 'AL']


,GEOID Census Tract,State,Census Tract Name,Population,Land Area Sq Miles,Population Density
1,01001020100,AL,"Census Tract 201, Autauga County, Alabama",1775,3.79,468.34
2,01001020200,AL,"Census Tract 202, Autauga County, Alabama",2055,1.28,1605.47
3,01001020300,AL,"Census Tract 203, Autauga County, Alabama",3216,2.06,1561.17
4,01001020400,AL,"Census Tract 204, Autauga County, Alabama",4246,2.46,1726.02
5,01001020501,AL,"Census Tract 205.01, Autauga County, Alabama",4322,2.40,1800.83
...,...,...,...,...,...,...
1433,01133965601,AL,"Census Tract 9656.01, Winston County, Alabama",2077,95.47,21.76
1434,01133965602,AL,"Census Tract 9656.02, Winston County, Alabama",3128,86.52,36.15
1435,01133965700,AL,"Census Tract 9657, Winston County, Alabama",4693,28.50,164.67
1436,01133965800,AL,"Census Tract 9658, Winston County, Alabama",4057,71.24,56.95


There we have it - a DataFrame complete with population density calculated per square mile for every census tract in the country!

In [21]:
# Park Slope Brooklyn NY
df_census_pop[df_census_pop['GEOID Census Tract'] == '36047016500']

,GEOID Census Tract,State,Census Tract Name,Population,Land Area Sq Miles,Population Density
51081,36047016500,NY,"Census Tract 165, Kings County, New York",5080,0.07,72571.43


And just for fun...a quick glimpse of our own neighborhood in Brooklyn. (It doesn't actually feel that crowded.)